# EA

In [2]:
import os 
import pandas as pd
import numpy as np

In [3]:
os.getcwd()

'/mnt/z/av/jobathon'

### Read Data

In [165]:
path = './data/train.csv'
df = pd.read_csv(path, parse_dates= ['MMM-YY', 'Dateofjoining', 'LastWorkingDate'], dayfirst=False)
df.head(10)

,MMM-YY,Emp_ID,Age,Gender,City,Education_Level,Salary,Dateofjoining,LastWorkingDate,Joining Designation,Designation,Total Business Value,Quarterly Rating
0,2016-01-01,1,28,Male,C23,Master,57387,2015-12-24,NaT,1,1,2381060,2
1,2016-02-01,1,28,Male,C23,Master,57387,2015-12-24,NaT,1,1,-665480,2
2,2016-03-01,1,28,Male,C23,Master,57387,2015-12-24,2016-03-11,1,1,0,2
3,2017-11-01,2,31,Male,C7,Master,67016,2017-11-06,NaT,2,2,0,1
4,2017-12-01,2,31,Male,C7,Master,67016,2017-11-06,NaT,2,2,0,1
5,2016-12-01,4,43,Male,C13,Master,65603,2016-12-07,NaT,2,2,0,1
6,2017-01-01,4,43,Male,C13,Master,65603,2016-12-07,NaT,2,2,0,1
7,2017-02-01,4,43,Male,C13,Master,65603,2016-12-07,NaT,2,2,0,1
8,2017-03-01,4,43,Male,C13,Master,65603,2016-12-07,NaT,2,2,350000,1
9,2017-04-01,4,43,Male,C13,Master,65603,2016-12-07,2017-04-27,2,2,0,1


In [169]:
def std_col_names(df):
    df.columns = df.columns.str.lower().str.strip().str.replace(' ', '_')
    df.rename(columns= {'mmm-yy': 'record_dt'}, inplace= True)
#     df[['record_dt', 'lastworkingdate']].dt.strftime("%m/%d/%y")
    return df

In [170]:
df = std_col_names(df)
# df["Date"].dt.strftime("%m/%d/%y")
df.head(3)


,record_dt,emp_id,age,gender,city,education_level,salary,dateofjoining,lastworkingdate,joining_designation,designation,total_business_value,quarterly_rating
0,2016-01-01,1,28,Male,C23,Master,57387,2015-12-24,NaT,1,1,2381060,2
1,2016-02-01,1,28,Male,C23,Master,57387,2015-12-24,NaT,1,1,-665480,2
2,2016-03-01,1,28,Male,C23,Master,57387,2015-12-24,2016-03-11,1,1,0,2


# Observing the data

In [171]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19104 entries, 0 to 19103
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   record_dt             19104 non-null  datetime64[ns]
 1   emp_id                19104 non-null  int64         
 2   age                   19104 non-null  int64         
 3   gender                19104 non-null  object        
 4   city                  19104 non-null  object        
 5   education_level       19104 non-null  object        
 6   salary                19104 non-null  int64         
 7   dateofjoining         19104 non-null  datetime64[ns]
 8   lastworkingdate       1616 non-null   datetime64[ns]
 9   joining_designation   19104 non-null  int64         
 10  designation           19104 non-null  int64         
 11  total_business_value  19104 non-null  int64         
 12  quarterly_rating      19104 non-null  int64         
dtypes: datetime64[ns

`LastWorkingDate` seems to be the only column with missing data, which can be taken care of  

In [172]:
df.describe()

,emp_id,age,salary,joining_designation,designation,total_business_value,quarterly_rating
count,19104.000000,19104.000000,19104.000000,19104.000000,19104.000000,1.910400e+04,19104.000000
mean,1415.591133,34.650283,65652.025126,1.690536,2.252670,5.716621e+05,2.008899
std,810.705321,6.264471,30914.515344,0.836984,1.026512,1.128312e+06,1.009832
min,1.000000,21.000000,10747.000000,1.000000,1.000000,-6.000000e+06,1.000000
25%,710.000000,30.000000,42383.000000,1.000000,1.000000,0.000000e+00,1.000000
50%,1417.000000,34.000000,60087.000000,1.000000,2.000000,2.500000e+05,2.000000
75%,2137.000000,39.000000,83969.000000,2.000000,3.000000,6.997000e+05,3.000000
max,2788.000000,58.000000,188418.000000,5.000000,5.000000,3.374772e+07,4.000000


### Adding LWD to relevant columns

In [173]:
df.lastworkingdate.isna().value_counts()

True     17488
False     1616
Name: lastworkingdate, dtype: int64

In [174]:
lwd_df = df[['emp_id', 'lastworkingdate']].dropna().rename(columns= {'lastworkingdate': 'lwd'})
lwd_df.head(2)

,emp_id,lwd
2,1,2016-03-11
9,4,2017-04-27


In [427]:
merged = df.merge(lwd_df, left_on= 'emp_id', right_on= 'emp_id', how= 'outer')
merged.head()

,record_dt,emp_id,age,gender,city,education_level,salary,dateofjoining,lastworkingdate,joining_designation,designation,total_business_value,quarterly_rating,lwd
0,2016-01-01,1,28,Male,C23,Master,57387,2015-12-24,NaT,1,1,2381060,2,2016-03-11
1,2016-02-01,1,28,Male,C23,Master,57387,2015-12-24,NaT,1,1,-665480,2,2016-03-11
2,2016-03-01,1,28,Male,C23,Master,57387,2015-12-24,2016-03-11,1,1,0,2,2016-03-11
3,2017-11-01,2,31,Male,C7,Master,67016,2017-11-06,NaT,2,2,0,1,NaT
4,2017-12-01,2,31,Male,C7,Master,67016,2017-11-06,NaT,2,2,0,1,NaT


In [428]:
merged.lwd.isna().value_counts()

False    10359
True      8745
Name: lwd, dtype: int64

### Calculate tenure

In [429]:
merged['tenure'] = ((merged.record_dt - merged.dateofjoining)/np.timedelta64(1, 'M')+1).astype('int')
merged.head(5)
# merged.tenure.dtype

,record_dt,emp_id,age,gender,city,education_level,salary,dateofjoining,lastworkingdate,joining_designation,designation,total_business_value,quarterly_rating,lwd,tenure
0,2016-01-01,1,28,Male,C23,Master,57387,2015-12-24,NaT,1,1,2381060,2,2016-03-11,1
1,2016-02-01,1,28,Male,C23,Master,57387,2015-12-24,NaT,1,1,-665480,2,2016-03-11,2
2,2016-03-01,1,28,Male,C23,Master,57387,2015-12-24,2016-03-11,1,1,0,2,2016-03-11,3
3,2017-11-01,2,31,Male,C7,Master,67016,2017-11-06,NaT,2,2,0,1,NaT,0
4,2017-12-01,2,31,Male,C7,Master,67016,2017-11-06,NaT,2,2,0,1,NaT,1


### Evaluate months before attrition

In [430]:
merged.lwd.fillna(value=pd.to_datetime('1/1/2015'), inplace=True)


In [431]:
merged.head(5)

,record_dt,emp_id,age,gender,city,education_level,salary,dateofjoining,lastworkingdate,joining_designation,designation,total_business_value,quarterly_rating,lwd,tenure
0,2016-01-01,1,28,Male,C23,Master,57387,2015-12-24,NaT,1,1,2381060,2,2016-03-11,1
1,2016-02-01,1,28,Male,C23,Master,57387,2015-12-24,NaT,1,1,-665480,2,2016-03-11,2
2,2016-03-01,1,28,Male,C23,Master,57387,2015-12-24,2016-03-11,1,1,0,2,2016-03-11,3
3,2017-11-01,2,31,Male,C7,Master,67016,2017-11-06,NaT,2,2,0,1,2015-01-01,0
4,2017-12-01,2,31,Male,C7,Master,67016,2017-11-06,NaT,2,2,0,1,2015-01-01,1


In [432]:
# def eval_t_attr(df):
#     for idx in range(len(df)):
#         if type(df.iloc[idx]['lwd']) == type(pd.NaT):
#             pass
#         else:
#             df.iloc[idx]['t_attr'] = ((df.lwd - df.record_dt)/np.timedelta64(1, 'M')).astype('int')
#     return df

In [433]:
merged['t_attr'] = ((merged.lwd - merged.record_dt)/np.timedelta64(1, 'M')).astype('int')

merged['attr_risk'] = merged['t_attr'].apply(lambda x: 0 if (x<0 or x>6) else 1)

In [434]:
merged['grade_chg_join'] = merged['designation'] - merged['joining_designation'] 

In [435]:
merged.head()

,record_dt,emp_id,age,gender,city,education_level,salary,dateofjoining,lastworkingdate,joining_designation,designation,total_business_value,quarterly_rating,lwd,tenure,t_attr,emp_risk,grade_chg_join
0,2016-01-01,1,28,Male,C23,Master,57387,2015-12-24,NaT,1,1,2381060,2,2016-03-11,1,2,1,0
1,2016-02-01,1,28,Male,C23,Master,57387,2015-12-24,NaT,1,1,-665480,2,2016-03-11,2,1,1,0
2,2016-03-01,1,28,Male,C23,Master,57387,2015-12-24,2016-03-11,1,1,0,2,2016-03-11,3,0,1,0
3,2017-11-01,2,31,Male,C7,Master,67016,2017-11-06,NaT,2,2,0,1,2015-01-01,0,-34,0,0
4,2017-12-01,2,31,Male,C7,Master,67016,2017-11-06,NaT,2,2,0,1,2015-01-01,1,-34,0,0


In [438]:
q2_df = merged.sample(frac= 0.75, replace= True, random_state= 1).groupby(by= 'record_dt').agg({'salary': ['median']})
q2_df.reset_index(inplace=True)


In [439]:
q2_df

,record_dt,salary
,,median
0,2016-01-01,55187.5
1,2016-02-01,55662.0
2,2016-03-01,54491.0
3,2016-04-01,56813.0
4,2016-05-01,56242.0
5,2016-06-01,56666.0
6,2016-07-01,59094.5
7,2016-08-01,60539.5
8,2016-09-01,62931.0


In [397]:
# merged.merge(q2_df, left_on= 'record_dt', right_on= 'record_dt', how= 'outer')

In [440]:
merged = merged.merge(q2_df, left_on= 'record_dt', right_on= 'record_dt', how= 'outer').sort_values(['emp_id', 'record_dt']).reset_index(drop= True)


/home/adawsl/anaconda3/envs/mlcamp/lib/python3.8/site-packages/pandas/core/frame.py:9186: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (1 levels on the left,2 on the right)
  return merge(
/home/adawsl/anaconda3/envs/mlcamp/lib/python3.8/site-packages/pandas/core/generic.py:4150: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


In [456]:
merged.rename({"[('salary', 'median')]": 'median_salary'}, inplace= True)

In [457]:
merged.head()

,record_dt,emp_id,age,gender,city,education_level,salary,dateofjoining,lastworkingdate,joining_designation,designation,total_business_value,quarterly_rating,lwd,tenure,t_attr,emp_risk,grade_chg_join,"(salary, median)"
0,2016-01-01,1,28,Male,C23,Master,57387,2015-12-24,NaT,1,1,2381060,2,2016-03-11,1,2,1,0,55187.5
1,2016-02-01,1,28,Male,C23,Master,57387,2015-12-24,NaT,1,1,-665480,2,2016-03-11,2,1,1,0,55662.0
2,2016-03-01,1,28,Male,C23,Master,57387,2015-12-24,2016-03-11,1,1,0,2,2016-03-11,3,0,1,0,54491.0
3,2017-11-01,2,31,Male,C7,Master,67016,2017-11-06,NaT,2,2,0,1,2015-01-01,0,-34,0,0,64444.0
4,2017-12-01,2,31,Male,C7,Master,67016,2017-11-06,NaT,2,2,0,1,2015-01-01,1,-34,0,0,65599.5


In [473]:
merged['q2_delta'] = merged['salary'] - merged[('salary', 'median')]

In [ ]:
cols= [
'age',
 'gender',
 'city',
 'education_level',
 'salary',
'total_business_value',
 'quarterly_rating',
'tenure',
 'grade_chg_join',
 ('salary', 'median')
]

target = 'attr_risk'

## References
- [pd.merge](https://pandas.pydata.org/docs/reference/api/pandas.merge.html#pandas-merge)
- [time in days to months](https://stackoverflow.com/questions/42822768/pandas-number-of-months-between-two-dates/42822819)
- [multiindex columns](https://towardsdatascience.com/accessing-data-in-a-multiindex-dataframe-in-pandas-569e8767201d)
- 
- 
- 
- 
-